# Topology with 2 networks, 1 containing all the servers and the other containing the client

## Import the FABlib library 

In [1]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config();

Orchestrator,orchestrator.fabric-testbed.net
Credential Manager,cm.fabric-testbed.net
Core API,uis.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,656d5ba5-76e6-44f2-9693-f65937c6de5e
Bastion Host,bastion.fabric-testbed.net
Bastion Username,comunidadoscar_0000173468
Bastion Private Key File,/home/fabric/work/fabric_config/fabric-bastion-key
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Sites to avoid,


## Creating the experiment slice

In [2]:
slice_name = 'CDN_Test_V3'
#[site1, site2] = fablib.get_random_sites(count=2)
sites = [site1, site2] = ['DALL', 'MAX']
print(f"Sites: {site1}, {site2}")

Sites: DALL, MAX


### Creating Slice

In [3]:
slice = fablib.new_slice(name=slice_name)

#### Adding Routers

In [4]:
subnet1 = IPv4Network("192.168.1.0/24")
gateway1 = subnet1[1]

net1 = slice.add_l2network(name='net1', subnet=subnet1, gateway=gateway1)

router1 = slice.add_node(name='LAN-A', site=site1, cores=2, ram=8, disk=10, image='default_ubuntu_22', host=f'{site1.lower()}-w1.fabric-testbed.net')
router1_local_iface = router1.add_component(model='NIC_Basic', name='nic_router1').get_interfaces()[0]

router1_local_iface.set_mode('config')
net1.add_interface(router1_local_iface)
router1_local_iface.set_ip_addr(gateway1);

In [5]:
subnet2 = IPv4Network("192.168.2.0/24")
gateway2 = subnet2[1]

net2 = slice.add_l2network(name='net2', subnet=subnet2, gateway=gateway2)

router2 = slice.add_node(name='LAN-B', site=site2, cores=2, ram=8, disk=10, image='default_ubuntu_22', host=f'{site2.lower()}-w1.fabric-testbed.net')
router2_local_iface = router2.add_component(model='NIC_Basic', name='nic_router2').get_interfaces()[0]


router2_local_iface.set_mode('config')
net2.add_interface(router2_local_iface)
router2_local_iface.set_ip_addr(gateway2);

#### Adding Server nodes to site 1


In [6]:
site_server_count = 3
for i, site in enumerate([site1]):
    print(f"Adding nodes to {site}")
    for node_num in range(site_server_count):
        node_name = f"server_{node_num+1}"
        print(f"Adding {node_name}")
        
        node = slice.add_node(name=node_name, site=site, cores=2, ram=8, disk=10, image='default_ubuntu_22', host=f'{site.lower()}-w2.fabric-testbed.net')
        iface = node.add_component(model='NIC_Basic', name='nic_local').get_interfaces()[0]
        network = slice.get_network(name='net1')
        
        network.add_interface(iface)
        
        iface.set_mode('auto')
        node.add_route(subnet=IPv4Network('192.168.0.0/16'), next_hop=network.get_gateway())

Adding nodes to DALL
Adding server_1
Adding server_2
Adding server_3


#### Adding Client node to site 2

In [7]:
site_node_count = 1
for i, site in enumerate([site2]):
    print(f"Adding nodes to {site}")
    for node_num in range(site_node_count):
        node_name = f"client_{node_num+1}"
        print(f"Adding {node_name}")
        node = slice.add_node(name=node_name, site=site, cores=2, ram=8, disk=10, image='default_ubuntu_22', host=f'{site.lower()}-w2.fabric-testbed.net')
        iface = node.add_component(model='NIC_Basic', name='nic_local').get_interfaces()[0]
        network = slice.get_network(name='net2')
        network.add_interface(iface)
        
        iface.set_mode('auto')
        node.add_route(subnet=IPv4Network('192.168.0.0/16'), next_hop=network.get_gateway())

Adding nodes to MAX
Adding client_1


#### Linking Router 1 and Router 2

In [8]:
link_info = {}
link_subnet = IPv4Network("10.10.100.0/24")
ip_addr1=link_subnet[1]
ip_addr2=link_subnet[2]

link = slice.add_l2network(name='router1_router2', subnet=link_subnet)

iface1 = router1.add_component(model='NIC_Basic', name=f'router1-router2').get_interfaces()[0]
iface1.set_mode('config')
link.add_interface(iface1)
iface1.set_ip_addr(ip_addr1)
iface1_dev = f"{{{{ interfaces['{iface1.get_name()}'].dev }}}}"

iface2 = router2.add_component(model='NIC_Basic', name=f'router2-router1').get_interfaces()[0]
iface2.set_mode('config')
link.add_interface(iface2)
iface2.set_ip_addr(ip_addr2)    
iface2_dev = f"{{{{ interfaces['{iface2.get_name()}'].dev }}}}"

#### Lets submit the slice now

In [9]:
slice.submit();


Retry: 10, Time: 335 sec


ID,63f85be2-6207-4a10-9dcb-684ee4cc2b0a
Name,CDN_Test_V3
Lease Expiration (UTC),2024-05-20 02:59:10 +0000
Lease Start (UTC),2024-05-19 02:59:12 +0000
Project ID,656d5ba5-76e6-44f2-9693-f65937c6de5e
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
285eb3fc-dd14-49b4-99be-428c51af42ab,LAN-A,2,8,10,default_ubuntu_22,qcow2,dall-w1.fabric-testbed.net,DALL,ubuntu,2001:400:a100:3000:f816:3eff:fe8f:11f,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3000:f816:3eff:fe8f:11f,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
6cde3a1c-878c-4a67-8c70-cee27f163c88,LAN-B,2,8,10,default_ubuntu_22,qcow2,max-w1.fabric-testbed.net,MAX,ubuntu,2001:468:c00:ffc4:f816:3eff:fed1:21d6,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:468:c00:ffc4:f816:3eff:fed1:21d6,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
ab803e7f-74f1-425b-998e-201ea2e16202,client_1,2,8,10,default_ubuntu_22,qcow2,max-w2.fabric-testbed.net,MAX,ubuntu,2001:468:c00:ffc4:f816:3eff:fea3:f642,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:468:c00:ffc4:f816:3eff:fea3:f642,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
60bcb059-984e-405c-b886-5d1d99e30991,server_1,2,8,10,default_ubuntu_22,qcow2,dall-w2.fabric-testbed.net,DALL,ubuntu,2001:400:a100:3000:f816:3eff:fea8:c970,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3000:f816:3eff:fea8:c970,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
f00623ff-5dd7-4693-a34b-f623827e93c4,server_2,2,8,10,default_ubuntu_22,qcow2,dall-w2.fabric-testbed.net,DALL,ubuntu,2001:400:a100:3000:f816:3eff:fed7:4608,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3000:f816:3eff:fed7:4608,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
13f985d2-ae18-4aab-9a1f-94c86e8ef593,server_3,2,8,10,default_ubuntu_22,qcow2,dall-w2.fabric-testbed.net,DALL,ubuntu,2001:400:a100:3000:f816:3eff:fe94:191c,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3000:f816:3eff:fe94:191c,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
e9fcbb4d-91f8-423f-8b10-2ebe9f264552,net1,L2,L2Bridge,DALL,192.168.1.0/24,192.168.1.1,Active,
5711e270-84d5-4135-8d8b-de4293f50234,net2,L2,L2Bridge,MAX,192.168.2.0/24,192.168.2.1,Active,
02cd5013-befb-4559-87e0-67fe891d2dd5,router1_router2,L2,L2STS,None,10.10.100.0/24,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node
LAN-A-nic_router1-p1,p1,LAN-A,net1,100,config,,4A:BB:14:67:C6:EF,enp7s0,enp7s0,192.168.1.1,6
LAN-A-router1-router2-p1,p1,LAN-A,router1_router2,100,config,,4E:86:DC:C4:34:50,enp8s0,enp8s0,10.10.100.1,6
LAN-B-nic_router2-p1,p1,LAN-B,net2,100,config,,06:40:B6:27:88:26,enp8s0,enp8s0,192.168.2.1,6
LAN-B-router2-router1-p1,p1,LAN-B,router1_router2,100,config,,02:9B:AE:2A:21:43,enp7s0,enp7s0,10.10.100.2,6
server_1-nic_local-p1,p1,server_1,net1,100,auto,,06:03:79:10:FC:85,enp7s0,enp7s0,192.168.1.3,4
server_2-nic_local-p1,p1,server_2,net1,100,auto,,06:02:A0:65:15:E6,enp7s0,enp7s0,192.168.1.4,4
server_3-nic_local-p1,p1,server_3,net1,100,auto,,06:5C:8B:7E:54:F6,enp7s0,enp7s0,192.168.1.2,4
client_1-nic_local-p1,p1,client_1,net2,100,auto,,02:53:47:22:66:F8,enp7s0,enp7s0,192.168.2.2,6



Time to print interfaces 335 seconds


## Install Apache on all servers

In [10]:
slice = fablib.get_slice(name=slice_name)
for node_num in range(site_server_count):
        node_name = f"server_{node_num+1}"
        print(f"installing apache on {node_name}")
        server = slice.get_node(name=node_name)
        try:
            print("Installing Apache...")
            stdout, stderr = server.execute("sudo apt-get update && sudo apt-get install -y apache2")
            print("Finished installing Apache.")
            
        except Exception as e:
            print(f"Exception: {e}")



installing apache on server_1
Installing Apache...
Hit:1 http://nova.clouds.archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://nova.clouds.archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:4 http://nova.clouds.archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:5 http://nova.clouds.archive.ubuntu.com/ubuntu jammy/universe amd64 Packages [14.1 MB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1467 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/main Translation-en [252 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1866 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/restricted Translation-en [317 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [852 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe Translation-en [164 kB]
Get:12 http://secur

## Install wireshark and tshark on routers

## Install HAProxy on router that is connected to the servers

## Delete the Slice

Please delete your slice when you are done with your experiment.

In [82]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Exception: {e}")